<a href="https://colab.research.google.com/github/darshan12x/projects/blob/mainbranch/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import tweepy
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
import numpy as np 
import matplotlib.pyplot as plt 
import string
import nltk
import warnings
import seaborn as sns
warnings.filterwarnings("ignore", category=DeprecationWarning)
nltk.download('vader_lexicon')

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [0]:
#My Twitter API Authentication Variables
consumer_key = 'VTgswlznw8Dzbj42HaL98NOAs'
consumer_secret = 'QDR7gDKuUzc6T2I6R96FZ449GdWXNrELrcAdK8S1GZEutd0DTp'
access_token = '957181535483908096-GdXFwmb1AFa0gmkhNwsDChpIOw2OY8r'
access_token_secret = '1VouYOsjd9TVK0cMksF4yHqZU3UOVTzg8KSxH2VtnAj9c'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [92]:
tweets = api.search('Irfan Khan', count=200)

# 'metadata': {'iso_language_code': 'en'
# 'location': ''
# 'followers_count': 41
# 'verified': False
# 'retweeted': False

data = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

display(data.head(10))

,Tweets
0,RT @KBhatia22: RIP Irfan Khan.. You live on in...
1,RT @Jimin_Serenity: #IrrfanKhan There are very...
2,RT @YUVSTRONG12: I know the journey I know the...
3,RT @YUVSTRONG12: I know the journey I know the...
4,RT @Payal_Rohatgi: Ram Ram ji 🙏 How I respecte...
5,It's hard to believe that you have left us\nI ...
6,RT @iamsrk_brk: Irfan Khan a struggler who bec...
7,"Rest In Peace, Irfan Khan (1966-2020). https:/..."
8,"First I saw #IrrfanKhan on silver screen, I kn..."
9,@SrBachchan I Miss You irfan khan Sir


In [87]:
# Alternate func
# regex to strips URL, punctuations, usernames and any non-alphanumeric chars. Also separates words with single space. 
""" 
def clean_tweet(tweet):
  return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",tweet).split())

data['tidy_tweet_v1'] = pd.DataFrame(data=[clean_tweet(tweet.text) for tweet in tweets], columns=['Tweets'])
data['tidy_tweet_v1'] = data['tidy_tweet_v1'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
display(data.head(10))

"""

' \ndef clean_tweet(tweet):\n  return \' \'.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\\w+:\\/\\/\\S+)"," ",tweet).split())\n\ndata[\'tidy_tweet_v1\'] = pd.DataFrame(data=[clean_tweet(tweet.text) for tweet in tweets], columns=[\'Tweets\'])\ndata[\'tidy_tweet_v1\'] = data[\'tidy_tweet_v1\'].apply(lambda x: \' \'.join([w for w in x.split() if len(w)>2]))\ndisplay(data.head(10))\n'

In [89]:
# tidy up tweets
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

# remove twitter handles (@user)
data['tidy_tweet'] = np.vectorize(remove_pattern)(data['Tweets'], "@[\w]*")

# remove links
data['tidy_tweet'] = np.vectorize(remove_pattern)(data['tidy_tweet'],"(\w+:\/\/\S+)")

# remove special characters, numbers, punctuations
data['tidy_tweet'] = data['tidy_tweet'].str.replace("([^0-9A-Za-z \t])", " ")

# removing short words
data['tidy_tweet'] = data['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

display(data.head(10))

,Tweets,tidy_tweet
0,"RT @StupidLarka420: Legends never die, they're...",Legends never die they always our heart Irfan ...
1,RT @AmitShah: Anguished over the sad news of I...,Anguished over the sad news Irfan Khan demise ...
2,RT @AmitShah: Anguished over the sad news of I...,Anguished over the sad news Irfan Khan demise ...
3,RT @Rashamians_Adda: This Is Lit!\nThis Is Wha...,This Lit This What Has Earned All gonna Miss H...
4,Completely shocked to hear of the demise of on...,Completely shocked hear the demise one the mos...
5,RT @dhruv_rathee: Heartbreaking to hear about ...,Heartbreaking hear about Irfan Khan death was ...
6,RT @Imanoop89: Irfan Khan - an actor beyond co...,Irfan Khan actor beyond compare May always rem...
7,நடிகர் இர்ஃபான் கான் மரணம் - சோகத்தில் ரசிகர்க...,IrrfanKhan Cinema News4Tamil
8,INNA LILLAHI WA INNA ILAYEHI Raji'un \n\nRip i...,INNA LILLAHI INNA ILAYEHI Raji Rip irfan khan ...
9,RT @LambaAlka: Irfan Khan the Actor is no more...,Irfan Khan the Actor more RIP IrfanKhan


In [82]:
sid = SentimentIntensityAnalyzer()
listy = []

for index, row in data.iterrows():
  ss = sid.polarity_scores(row["tidy_tweet_v1"])
  listy.append(ss)
  
se = pd.Series(listy)
data['polarity'] = se.values

display(data.head(100))

,Tweets,tidy_tweet_v1,tidy_tweet_v2,polarity
0,RT @PrakashJavdekar: Irfan Khan was a versatil...,Irfan Khan was versatile actor Sorry hear abou...,Irfan Khan was versatile actor Sorry hear abou...,"{'neg': 0.062, 'neu': 0.622, 'pos': 0.316, 'co..."
1,RT @LahriSunil: Very sad to hear sudden demise...,Very sad hear sudden demise one the finest act...,Very sad hear sudden demise one the finest act...,"{'neg': 0.123, 'neu': 0.652, 'pos': 0.225, 'co..."
2,@AmitShah Anguished over the sad news of Irfan...,Anguished over the sad news Irfan Khan demise ...,Anguished over the sad news Irfan Khan demise ...,"{'neg': 0.296, 'neu': 0.704, 'pos': 0.0, 'comp..."
3,RT @AmitShah: Anguished over the sad news of I...,Anguished over the sad news Irfan Khan demise ...,Anguished over the sad news Irfan Khan demise ...,"{'neg': 0.238, 'neu': 0.645, 'pos': 0.117, 'co..."
4,"প্রয়াত Irfan Khan sir, One of my favorite act...",Irfan Khan sir One favorite actor Life Metro P...,Irfan Khan sir One favorite actor Life Metro P...,"{'neg': 0.0, 'neu': 0.824, 'pos': 0.176, 'comp..."
...,...,...,...,...
95,RT @Gul_3_Lala: Irfan khan died due to cancer ...,Lala Irfan khan died due cancer was such good ...,Irfan khan died due cancer was such good perso...,"{'neg': 0.407, 'neu': 0.491, 'pos': 0.102, 'co..."
96,https://t.co/eKQ15bYYaz\n#RIPIrrfanKhan\n#irfa...,RIPIrrfanKhan irfankhandeath,#RIPIrrfanKhan #irfankhandeath,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
97,"Twitterati, please stop tagging Irfan Pathan, ...",Twitterati please stop tagging Irfan Pathan in...,Twitterati please stop tagging Irfan Pathan in...,"{'neg': 0.133, 'neu': 0.727, 'pos': 0.139, 'co..."
98,Inna’alilahi Wa Inna’alilahi Ra’ajioon... \nRe...,Inna alilahi Inna alilahi ajioon Rest Peace Ir...,Inna alilahi Inna alilahi ajioon Rest Peace Ir...,"{'neg': 0.0, 'neu': 0.811, 'pos': 0.189, 'comp..."


In [56]:
#  tokenize all the cleaned tweets in our dataset
tokenized_tweet = data['tidy_tweet'].apply(lambda x: x.split())

tokenized_tweet.head()

0    [What, was, your, favourite, Film, character, ...
1    [One, favourite, actor, Rest, Peace, Irfan, Khan]
2    [Such, sad, sad, news, Actor, Irfan, Khan, pas...
3    [artist, who, was, blessed, God, like, Bali, t...
4    [Heartbreaking, hear, about, Irfan, Khan, deat...
Name: tidy_tweet, dtype: object